In [3]:
from colorama import Fore, Back, Style

#STEP 1: IMPORT DES DONNEES LINGUISTIQUES WIKIPEDIA DE TEST
#wikipedia data languages reading
#test avec les dataframes de pandas

#When reading the txt file with pandas' read_csv, some line loss occurred.
#The reason is that there is a single English quotation mark in a line, causing the \n line break to be invalid.
#Multiple lines are concatenated until the next single quote is encountered
#add option quoting=3 to tackle the issue :QUOTE_NONE

import pandas as pd
import numpy as np

dflabel = pd.read_csv('input/labels.csv', delimiter = ';')
dfx1 = pd.read_table('input/x_train.txt', delimiter = None, header=None, encoding='utf-8', quoting=3)
dfy1 = pd.read_table('input/y_train.txt', delimiter = None, header=None, encoding='utf-8', quoting=3)
dfx2 = pd.read_table('input/x_test.txt', delimiter = None, header=None, encoding='utf-8', quoting=3)
dfy2 = pd.read_table('input/y_test.txt', delimiter = None, header=None, encoding='utf-8', quoting=3)

#row number quick checking
#print(len(dfx1.index))
#print(len(dfx2.index))
#print(len(dfy1.index))
#print(len(dfy2.index))
#print(len(dflabel.index))

#step 2: EXTRACTION DES PARAGRAPHES DES 5 LANGUES LES PLUS PARLEES
#5 most spoken languages selection
#https://en.wikipedia.org/wiki/List_of_languages_by_total_number_of_speakers
#english / mandarin / hindi / spanish / standard arabic

#sentences and language merging
df1 = pd.concat([dfx1, dfy1], axis=1)
df1.columns=["para","langcod"]
df2 = pd.concat([dfy2, dfy2], axis=1)
df2.columns=["para","langcod"]

#5 languages extraction from labels
dflabel5mml=dflabel.loc[(dflabel['English']=='Arabic')|
                        (dflabel['English']=='English')|
                        (dflabel['English']=='Hindi')|
                        (dflabel['English']=='Spanish')|
                        (dflabel['English']=='Standard Chinese')]
dflabel5mml2=dflabel5mml.rename(columns={'Label':'langcod'})
dflabel5mml3=dflabel5mml2[['langcod','English']]

#sentences selection by most spoken languages-> merging
df1s = pd.merge(df1, dflabel5mml3, how="inner", on=["langcod"])
df2s = pd.merge(df2, dflabel5mml3, how="inner", on=["langcod"])

#row number quick checking
#print(len(df1s.index))
#print(len(df2s.index))

#step 3: EXTRACTION DE N PARAGRAPHES ALEATOIRES POUR NE PAS EXPLOSER LES COMPTEURS AZURE
#extraction of n random sentences form dataframe
import random

def rand_shrink_df(df):
    
    size = input("Input a number of paragraphs, bigger than 5, that you want to test ,\
keep in mind that azure is not free > ")
    sizeint = int(size)
    #Generate random numbers
    #randomlist = random.sample(range(0, len(df.index)-1), sizeint)
    randomlist = random.sample(range(0, 500), sizeint)
    #print(randomlist)
    #trick to get all the language
    factor = [(i % 5)*500 for i in range(sizeint)]
    #print(factor)
    #operation of 2 list with zip
    product = [x+y for x,y in zip(randomlist,factor)]
    #print(product)



    for i in range(sizeint):
                            #attention pour que le dataframe reste en row, penser aux double crochets
        dfa=df.iloc[[product[i]],:]
        #print(i)
        if i==0 :
            dfaa=dfa
            #print(dfaa)
            #print(len(dfaa.index))
        else:
            dfaa=dfaa.append(dfa)
            #print(len(dfa.index))
    return dfaa

df2az=rand_shrink_df(df1s)
#print(len(df2az.index))
#print(df2az)


###################################################################
###################################################################


#step 4: AUTHENTIFICATION DU CLIENT AU SERVICE cognitive-services
#authenticate the client
#azure key vault (coffre fort)  to securely store keys

#step 5: DETECTION DU LANGAGE VIA L'IA TEXT ANALYTICS D'AZURE ou TRANSLATOR DETECT D'AZURE AU CHOIX
#language detection by text analytics
import os
import cmd
import requests, uuid, json

from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def azure_language_detection(df):
    
    #https://docs.microsoft.com/en-us/python/api/overview/azure/identity-readme?view=azure-python
    credential = DefaultAzureCredential()
    #get the needed environmental variables to connect to the keyvault
    keyVaultName = os.getenv('KEY_VAULT_NAME')
    KVUri = f"https://{keyVaultName}.vault.azure.net"
    client = SecretClient(vault_url=KVUri, credential=credential)

    #get the key frome the azure key vault to connect to translator
    secretName1 = "keytranslator"
    key1=client.get_secret(secretName1)
     # Add your subscription key and endpoint
    subscription_key1 = key1.value
    endpoint1 = "https://api.cognitive.microsofttranslator.com/detect?api-version=3.0"
    location = "francecentral"
    
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key1,
        'Content-type': 'application/json',
        'Ocp-Apim-Subscription-Region': location,
        'X-ClientTraceId': str(uuid.uuid4())
    }

    
    def authenticate_client_ta():
        secretName2 = "keytextanalytics"
        endpoint2 = "https://ouddane.cognitiveservices.azure.com/"
        key2=client.get_secret(secretName2)
        ta_credential = AzureKeyCredential(key2.value)
        text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint2, 
            credential=ta_credential) 
        return text_analytics_client
    client = authenticate_client_ta()
    
    srv= input("Do you want to use Text Analytics or Translator Detect? write TA or TD > ")
    own= input("Do you want to test your own sentence? write Y or N > ")
    
    if srv=="TA":
        if own=="Y":
            doc= [input("paste your sentence > ")]
            response = client.detect_language(documents = doc, country_hint = "")[0]
            print(Fore.GREEN + "According to AZURE TEXT ANALYTICS, the language of your sentence is ",response.primary_language.name )
        else:
            df["azure"] = ""
            df["flag"]= 0
            for i in range(len(df.index)):
                try:
                    documents = [df.iloc[i,0]] 
                    response = client.detect_language(documents = documents, country_hint = "")[0]
                    df.iloc[i,3]=response.primary_language.name
                    if (df.iloc[i,3]== "Chinese_Traditional") | (df.iloc[i,3]== "Chinese_Simplified"):
                        df.iloc[i,3]= "Standard Chinese"
                    if df.iloc[i,3]== df.iloc[i,2]:
                        df.iloc[i,4]=1
                    else:
                        df.iloc[i,4]=0
                except Exception as err:
                    print("Encountered exception. {}".format(err))
                
            print(Fore.GREEN + "TEXT ANALYTICS detection works at",df["flag"].mean()*100,"%")
            if df["flag"].mean()<1:
                print("The error comes maybe from the input data: wrong flag or several languages in the same sentence")
                df0=df.loc[(df['flag']==0)]
                print(df0)
            print(Style.RESET_ALL)
            print(df)
            
    elif srv=="TD":
        if own=="Y":
            doc= input("paste your sentence > ")
            # You can pass more than one object in json body.
            body = [{
                'text': doc
            }]
            ## lib requests
            ## post sends a POST request (http) to the specified url and is used when you want to send some data to the server
            request = requests.post(endpoint1, headers=headers, json=body)
            ##response.json() returns a JSON object of the result / Whenever we make a request to a specified URI through Python, it returns a response object
            response = request.json()
        
            #json.dumps() function converts a Python object into a json string
            response2 = json.dumps(response)
            #json.loads() method can be used to parse a valid JSON string and convert it into a Python Dictionary
            dictresponse = json.loads(response2)[0]

            #print(dictresponse)
            print(Fore.GREEN + "According to AZURE TRANSLATOR DETECT, the language of your sentence is ", dictresponse['language'])
        
        else:
            #init flag    
            df["azure"] = ""
            df["flag"]= 0
            df["azscore"]=0
            for i in range(len(df.index)):
               # try:
                doc = df.iloc[i,0] 
                body = [{
                       'text': doc
                 }]
                request = requests.post(endpoint1, headers=headers, json=body)
                response = request.json()
                response2 = json.dumps(response)
                dictresponse = json.loads(response2)[0]
                #print(dictresponse)
                df.iloc[i,3] = dictresponse['language']
                df.iloc[i,5] = dictresponse['score']
                if (((df.iloc[i,3]== "en") & (df.iloc[i,2]== "English"))|
                    ((df.iloc[i,3]== "ar") & (df.iloc[i,2]== "Arabic"))|
                    ((df.iloc[i,3]== "es") & (df.iloc[i,2]== "Spanish"))|
                    ((df.iloc[i,3]== "hi") & (df.iloc[i,2]== "Hindi"))|
                    ((df.iloc[i,3]== "zh-Hant") & (df.iloc[i,2]== "Standard Chinese"))|
                    ((df.iloc[i,3]== "zh-Hans") & (df.iloc[i,2]== "Standard Chinese"))):
                    df.iloc[i,4]=1
                else:
                    df.iloc[i,4]=0
               # except Exception as err:
               #     print("Encountered exception. {}".format(err))
                
            print(Fore.GREEN + "TRANSLATOR detection works at",df["flag"].mean()*100,"% according to our indicator and azure average confidence indicator is equal to",df["azscore"].mean()*100,"%")
            if df["flag"].mean()<1:
                print("The error comes maybe from the input data: wrong flag or several languages in the same sentence")
                #65417 de train.txt: plusieurs langues
                df0=df.loc[(df['flag']==0)]
                print(df0)
            print(Style.RESET_ALL)
            print(df)
azure_language_detection(df2az)


Input a number of paragraphs, bigger than 5, that you want to test ,keep in mind that azure is not free > 5
Do you want to use Text Analytics or Translator Detect? write TA or TD > TA
Do you want to test your own sentence? write Y or N > Y
paste your sentence > Nous déposons des cookies pour optimiser votre expérience utilisateur, améliorer la performance du site, analyser son traffic, vous permettre d'acheter des produits et services relatifs aux évènements sportifs qui vous intéressent, fournir les fonctionnalités des réseaux sociaux et vous montrer notre publicité ciblée, ou celle de nos partenaires.
According to AZURE TEXT ANALYTICS, the language of your sentence is  French
